<h1> Bayesian VAR estimation

## Structural Form Equation

$\
B y_t = \Gamma y_{t-1} + e_t
$

This represents a structural vector autoregressive (VAR) model, where matrix is:

$\
B =
\begin{bmatrix}
b_{11} & b_{12} \\
b_{21} & b_{22}
\end{bmatrix}, \quad
\Gamma =
\begin{bmatrix}
\gamma_{11} & \gamma_{12} \\
\gamma_{21} & \gamma_{22}
\end{bmatrix}
$

## Reduced Form Equation

By multiplying both sides of the structural equation by $\ B^{-1} $ :

$\
y_t = B^{-1} \Gamma y_{t-1} + B^{-1} e_t
$

This simplifies to:

$\
y_t = \Phi y_{t-1} + u_t
$

Where $ \ u_t = B^{-1} e_t $ is reduced-form shock


<h1> Load and Prepare Data

In [ ]:
data = pd.read_excel('Data.xlsx', sheet_name='VAR', usecols='A:C', skiprows=1).to_numpy()
def demean_data(data):
    return data - np.mean(data, axis=0)

Y = demean_data(data)
k = Y.shape[1]

<h1> Define MCMC Parameters

In [ ]:
n0 = 500  # Burn-in size
n1 = 2000  # MCMC sample size after burn-in
n = n0 + n1

<h1> Setting Priors

In [ ]:
# nu: Degrees of freedom for the prior on the residual covariance matrix 
# R0: Prior for the precision matrix 

nu = 10
R0 = inv(0.2 * np.eye(k)) / nu



# Priors for coefficients

pkk = p * k * k
b_ = np.zeros(pkk)  # Prior mean for vec phi(Φ),
var_ = np.eye(pkk)  # Prior variance for vec(Φ)

# Combine

Spec = {
    'b_': b_,
    'var_': var_,
    'p': p,
    'nu': nu,
    'R0': R0,
    'Y': Y,
    'mlag': mlag
}

<h1> Recursive VAR function

### In Bayesian VAR 

The posterior distributions for parameters:

- Autoregressive coefficients $ \Phi $
- Covariance matrix $ \Sigma_u $

If we assume **normal priors** for $ \Phi $ and **inverse-Wishart priors** for $ \Sigma_u $, their posterior distributions are **known and tractable**.

The inverse-Wishart prior is a standard choice for covariance matrices because it is conjugate to the multivariate normal likelihood.

Conjugacy ensures the posterior for $ \Sigma_u $ is also inverse-Wishart, making computations tractable.

MCMC algorithms can sample directly from these conditional distributions without explicitly calculating the likelihood.


In [ ]:
# Define Recursive VAR Function
def recursive_var(n0, n1, spec):
    """
    Function to estimate SVAR parameters using MCMC and compute impulse responses.
    """
    # Extract variables from spec
    Y = spec['Y']
    p = spec['p']
    k = Y.shape[1]  # Number of variables
    T = Y.shape[0]  # Number of observations
    
    # Prepare lagged Y matrix (X) and target Y (endogenous variables)
    X = np.hstack([Y[p - lag - 1:-lag - 1] for lag in range(p)])
    Y_target = Y[p:]  # Drop the first p rows
    
    # Reduced-form VAR estimation (OLS for initialization)
    B_hat = np.linalg.lstsq(X, Y_target, rcond=None)[0]  # OLS coefficients
    residuals = Y_target - X @ B_hat
    Sigma_u_hat = residuals.T @ residuals / (T - p)  # Residual covariance matrix
    
    # Cholesky decomposition for identification
    B_chol = cholesky(Sigma_u_hat, lower=True)  # Structural matrix
    
    # Initialize MCMC samples
    Phi_samples = []  # Autoregressive coefficients
    Sigma_u_samples = []  # Covariance matrices
    
    # MCMC loop
    for i in range(n0 + n1):
        # Sample Phi (coefficients) conditional on Sigma_u
        Sigma_u_inv = inv(Sigma_u_hat)
        Phi_post_var = inv(np.kron(Sigma_u_inv, X.T @ X) + inv(spec['var_']))
        Phi_post_mean = Phi_post_var @ (np.kron(Sigma_u_inv, X.T) @ Y_target.ravel() + inv(spec['var_']) @ spec['b_'])
        Phi = np.random.multivariate_normal(Phi_post_mean, Phi_post_var)
        
        # Reshape Phi back to VAR coefficient matrix
        Phi_matrix = Phi.reshape((k, p * k))
        
        # Sample Sigma_u (covariance matrix) conditional on Phi
        resid = Y_target - X @ Phi_matrix.T
        S = resid.T @ resid
        Sigma_u_inv_post = inv(spec['R0'] + S)
        Sigma_u = inv(np.random.wishart(df=spec['nu'] + T - p, scale=Sigma_u_inv_post))
        
        # Save post-burn-in samples
        if i >= n0:
            Phi_samples.append(Phi_matrix)
            Sigma_u_samples.append(Sigma_u)
    
    # Compute Impulse Responses
    impulse_responses = np.zeros((spec['mlag'], k, k))  # Horizon x Variables x Shocks
    for lag in range(spec['mlag']):
        Phi_cum = np.eye(k) if lag == 0 else Phi_samples[-1][:, :(lag * k)]
        impulse_responses[lag] = Phi_cum @ B_chol
    
    return impulse_responses, {"Phi_samples": Phi_samples, "Sigma_u_samples": Sigma_u_samples}



<h1> Run MCMC

In [ ]:
start_time = time.time()
ImpulseRespm, MHm = recursive_var(n0, n1, Spec)
end_time = time.time()

print(f"Elapsed time: {end_time - start_time:.2f} seconds")

# Inspect results
print(f"Impulse Responses Shape: {ImpulseRespm.shape}")